## **Exploratory analysis and prediction on the "TMDB 5000 Movie Dataset" dataset**

***Authors: Bava Flavio 4836427 , Ciarlo Francesco 4640121, Oldrini Edoardo 4055097***

**Abstract**

The following data analysis aims to study an approach for the production of a movie.<br><br>
This file is divided like so:
* Dataset checking and preparation
* Initial exploration of the dataset
* Raising hypotheses and subsequent verification
* Proposal for a predictive regression based on previous observations

**Importing libraries and datasets**

In [ ]:
#ciaoragazzi
import matplotlib.pyplot as plt # this is used for the plot the graph 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # used for plot interactive graph.
import numpy as np 
import plotly.offline as py
import json
from plotnine import ggplot,aes,facet_grid,labs,geom_col,theme_xkcd
py.init_notebook_mode(connected=True) #TODO ????
import plotly.graph_objs as go
import warnings #TODO ????
warnings.filterwarnings('ignore') #TODO ????
from pylab import rcParams #TODO ????
from ast import literal_eval

In [ ]:
Movies = pd.read_csv('tmdb_5000_movies.csv')

### **Exploratory analysis of the dataset**

Moving all columns with Json data into list

In [ ]:
#TODO: va aggiunto qualcosa per attorie  registi o si possono usare queste?
def get_name(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        return names
    return []

def get_ISO(x):
    if isinstance(x, list):
        isos = [i['iso_3166_1'] for i in x]
        return isos
    return []

Funzioni per passare da json a liste dei dati utili

In [ ]:
feat_to_manage = ['genres','keywords','production_countries','production_companies'] #aggiungere attori e registi

In [ ]:
for f in feat_to_manage:
    Movies[f] = Movies[f].apply(literal_eval)

In [ ]:
#Turn genres into list
Movies['genres'] = Movies['genres'].apply(get_name)
#Turn prod_countries into list
Movies['production_countries'] = Movies['production_countries'].apply(get_ISO)
#Turn prod_companies into list
Movies['production_companies'] = Movies['production_companies'].apply(get_name)
#Turn keywords into list
Movies['keywords'] = Movies['keywords'].apply(get_name)

Movies.head(1)

print("Generi hanno tipo ", type(Movies['genres'][0]))
print("Paesi produzione hanno tipo ", type(Movies['production_countries'][0]))


Check dataset dimension and an overview of the table itself

Droppiamo colonne inutili

In [ ]:
Movies.drop(['homepage','spoken_languages','original_title'],inplace=True,axis='columns')
Movies.shape

1. Controlliamo chei tipi siano giusti

In [ ]:
Movies.info()

I tipi sono coerenti

2. Controlliamo se ci sono dati mancanti

In [ ]:
print(Movies.isnull().sum())

ci sono alcuni valori nulli ma in colonne che potrebbero non essere significative

3. Controlliamo che i budget abbiano valori sensati

In [ ]:
#TODO possibile predizione buget come studio
print(Movies['budget'].describe())

Il valore minimo è 0, abbiamo quindi il dubbbio che ci siano budget con valori insiensati, per conoscenza del dominio applciativo decidiamo di porre il limite inferiore di 10k per il budget, verifichiamo quanti film sono sotto a questo valore.

Ci sono 1400 film con un valore di budget che non sembra avere senso, per evitare di eliminare tutti i dati relativi a questo film decidiamo di "segnarli" ponendo il budget ad un valore simbolico di -1 questi dati in modo da non prenderli in considerazione nelle statistiche ma non perdere altri dati interesanti.

In [ ]:
#Movies[Movies['budget'] <= 10000]['budget'] = np.nan
for row in Movies.index:
    if Movies.loc[row,'budget'] < 10000 and (Movies.loc[row, 'status']=='Released'):
        Movies.loc[row,'budget'] = np.nan
print(Movies[Movies['budget'] == np.nan].shape)

In [ ]:
print(Movies[Movies['budget']<10000]['id'].count())
#Movies.tail()

4. Controlliamo che i revenue abbiano valori sensati

In [ ]:
#TODO possibile predizione buget come studio
#Movies.tail()
print(Movies['revenue'].describe())

molti film hanno revenue nullo, che siano film ancora non usciti?

In [ ]:
print('Movies with 0$ revenues: ',Movies[Movies['revenue'] == 0].shape[0])
print('Movies not yet released', Movies[(Movies['revenue'] == 0) & (Movies['status']!= 'Released')].shape[0])

Su 1427 film che hanno revenue = 0 solo 7 sono ancora in fase di produzione, quindi 1427 film hanno un valore di revenue errato che procediamo a segnare, come visto precedentemente, con un valore simbolico di -1

In [ ]:
for row in Movies.index:
    if (Movies.loc[row, 'revenue'] == 0) and (Movies.loc[row, 'status']=='Released'):
        Movies.loc[row, 'revenue'] = -1

print('Movies without revenue infos: ',Movies[Movies['revenue'] == -1].shape[0])


3. Ulteriori aggiustamenti

Aggiungiamo due colonne legate al budget, una misura i ricavi sulla percentuale rispetto al budget, l'altra divide i film in 3 categorie, low budget, normali e blockbuster


1) Ricavi rispetto al budget


In [ ]:
def calculate_profit_perc(x):
    if (x.revenue>0) and (x.budget>0):
        return ((x.revenue-x.budget)/x.budget)*100
    #senza else mette i nan, altrimenti possiamo metterlo e ritornare -1

In [ ]:
#Movies = Movies.assign(profit_perc = lambda x: (((x.revenue-x.budget)/x.budget)*100) if ((x.revenue>0) and (x.budget>0)) else -1)
#print(Movies[Movies['budget'].isnull()])
Movies = Movies.assign(profit_perc = lambda x: x.budget)
for row in Movies.index:
    Movies.loc[row,'profit_perc'] =  calculate_profit_perc(Movies.loc[row])



2. Classificazioni film in low, med, high budget

usiamo describe per capire come impostare gli scaglioni

In [ ]:
Movies[Movies['budget']>0]['budget'].describe()

Low budget = 25% con budget più bassi<br>
High budget = 25% con budget più alti<br>
Medio = il restante 50%

In [ ]:
def budget_range(x):
    if x <= 0: #x==NaN
        return "no budget info"
    if x < 7.900000e+05:
        return "low"
    elif x >= 7.900000e+05 and x < 4.000000e+07:
        return "medium"
    else:
        return "high"

aggiungiamo una colonna che contiene l'etichetta relativa al budget

In [ ]:
Movies = Movies.assign(budget_class = lambda x: str(x.budget))
for row in Movies.index:
    Movies.loc[row,'budget_class'] =  budget_range(Movies.loc[row,'budget'])


In [ ]:
#TODO: perchè prendiamo solo i budget>10^7 e non tutti i non negativi?
#sns.kdeplot(data=Movies[Movies['budget'] >0], x='budget', shade = True)
sns.kdeplot(data=Movies[Movies['budget'] >= 4.000000e+07], x='budget', shade = True)
plt.legend(['Budget'])
plt.show()


In [ ]:
#TODO: la possiamo cancellare??
#Movies['budget'].describe()
#Movies[Movies['budget'] <= 7.900000e+05].shape
#Movies[(Movies['budget'] <= 1.500000e+07) & Movies['budget']<= 4.000000e+07].shape
#Movies[(Movies['budget'] >=   1.500000e+07) & (Movies['budget'] <= 4.000000e+07)].shape
#print(11 in range(0,10))
#Movies([Movies['budget'] >= 1.500000e+07] and (Movies['budget'] <= 4.000000e+07)).shape


Riteniamo importante che la media dei voti sia pesata con il numero di voti che la generano, per fare questo usiamo una formula consigliata dal celebre sito IMBD e togliamo le due colonne average e count per unirle in una che le metta insieme

In [ ]:
#TODO: potremmo anche fare questo lavoro sul dataset di partenza aggiungendo una colonna, metà avranno nan
C= Movies['vote_average'].mean()
C
m= Movies['vote_count'].quantile(0.5)

for i in Movies.index:
    if Movies.loc[i,'vote_count'] <= m:
        Movies.loc[i,'vote_count'] = np.nan
    else:
        pass

    
q_movies = Movies[['id','vote_count','vote_average']]


In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)
q_movies.drop(['vote_average','vote_count'],inplace=True,axis='columns')
#Merge dei dataframe,ora Movies ha anche la colonna Score",
Movies = pd.merge(Movies,q_movies,on='id',how='inner')
Movies.shape
   

Abbiamo eliminato metà dei film, quelli con troppi pochi voti per poter essere presi in considerazione per quanto riguarda il voto ricevuto.

Breve sguardo agli attributi quantitativi

In [ ]:
Movies.describe()

### **Cominciamo a fare qualche ipotesi**

**1. Dati qualitativi**

usiamo una funziona di libreria per portare genri, keyword, paesi di produzione e compagnie di produzione in un formato più semplice da gestire

******************************************************************************************************************************************
******************************************************************************************************************************************

## Dati qualitativi ##

Proviamo a definire delle funzioni per verificare la distribuzione di voti e soldoni fatti in base ai dati qualitativi

In [ ]:
#TODO: perchè è commentata? Non serve più?
#def get_name(x):
 #   if isinstance(x, list):
  #      names = [i['name'] for i in x]
   #     return names
    #return []

### Primo qualitativo: generi ###

1. definiamo funzione che dato un genere calcola la media dei profitti dei film che lo contengono

In [ ]:
def genre_average_profits(genre):
    sum = 0
    count = 0
    for row in Movies.index:
        if (genre in Movies.loc[row, 'genres']) and (Movies.loc[row,'profit_perc'] > 0):
            sum += Movies.loc[row, 'profit_perc']
            count+=1
    return count

#Movies['profit_perc'] = Movies.apply(lambda row: genre_average_profits(row['genres']), axis=1)

2. estraiamo dal dataset la lista dei generi presenti

In [ ]:
genres=[]
for row in Movies.index:
    _gen = Movies.loc[row,'genres']
    for g in _gen:
        if g not in genres:
            genres.append(g)

print("I generi presenti nel dataframe sono:")
for el in genres:
    print('\t',el)

3. calcoliamo e salviamo il profitto medio di ogni genere

In [ ]:
profits=[]
for g in genres:
    profits.append(genre_average_profits(g))
            
print("Profitto associato ad ogni genere:")
for i in range(0,len(genres)):
    print('\t',genres[i], " has a mean profit of ", profits[i])

Due generi hanno una media pari a 0, appaiano quindi solo nei film per cui non abbiamo info su budget e/o revenue, li eliminiamo quindi dalla lista dei generi

In [ ]:
genres.remove(genres[19])
genres.remove(genres[18])
profits.remove(profits[19])
profits.remove(profits[18])

3. Di seguito possiamo vedere come sono i profitti in base ai diversi generi dei film

In [ ]:
#TODO: riusciamo a farli in ordine decrescente sul grafico?
fig, ax = plt.subplots()
ax.bar(genres,profits)
fig.set_figwidth(27)
fig.set_figheight(13)
plt.xticks(rotation=45,fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("Genres", fontsize=20)
plt.ylabel("Profits", fontsize=20)
plt.show()

Per ottimizzare le funzioni di libreria sostituiamo ai valori dei profitti associati ai film con dati insufficienti a calcolarli il valore nan

In [ ]:
#TODO: si può mica fare da subito?
for row in Movies.index:
    if (Movies.loc[row, 'profit_perc'] <= 0) and (Movies.loc[row, 'status']=='Released'):
        Movies.loc[row, 'profit_perc'] = np.nan

### Secondo qualitativo: keywords ###

1. importiamo i profitti e le keywords associate solo a film su cui non abbiamo dati relativi ai profitti, precedentemente calcolate con una funzione pesante

In [ ]:
#key_profits = pd.read_csv('key_profits.csv')['0']
#bad_key_df = pd.read_csv('bad_key.csv')
#bad_key=bad_key_df['0']

2. estraiamo dal dataset la lista delle presenti

In [ ]:
keywords=[]
for row in Movies.index:
    _key = Movies.loc[row,'keywords']
    for k in _key:
        if g not in keywords:
            keywords.append(k)
print(len(keywords))

3. definiamo una funzione che data una keyword ne calcola la media dei profitti associati

In [ ]:
def keyword_average_profit(keywords):
    sum = 0
    count = 0
    for row in Movies.index:
        if (keywords in Movies.loc[row, 'keywords'] and Movies.loc[row, 'profit_perc'] >= 0):
            sum += Movies.loc[row, 'profit_perc']
            count+=1
    if count != 0:
        return sum/count
    else:
        return -1

4. ora per ogni keyword calcoliamo il profitto e salviamo quali keywords non hanno dati su budget o revenue

In [ ]:
#funzione pesantissima con hash
set_key = set(keywords)
key_profits={} #declared as hashmap
bad_keys=[]
for k in set_key:
    q = keyword_average_profit(k)
    if q != -1:
        key_profits[str(k)]=q
    else:
        bad_keys.append(k)

5. A questo punto castiamo l'array di tutte le keywords e di quelle da scartare in set, così da eleminare ogni duplicato e poter effettuare una differenza:

In [ ]:
bad_key_set = set(bad_keys)
set_key_cleaned = set_key - bad_key_set

print("Numero di keywords:\t",len(set_key))
print("Numero di keywords da scartare\t",len(bad_key_set))
print("Numero di keywords da analizzare:\t",len(set_key_cleaned))
print("Numero di profitti per le keywords calcolati\t",len(key_profits))

#pd.DataFrame(key_profits).to_csv("key_profits.csv")

In [ ]:
sorted_key_hash = sorted(key_profits.items(), key=lambda x: x[1])
print(sorted_key_hash)

6. visto il grande numero di keywords presenti nel dataset ci concentriamo su quelle più presenti

In [ ]:
from collections import Counter #per prendere le più presenti

best = 100 #numero di keywords da studiare 
_most_common_keys = Counter(keywords).most_common(best)
most_common_keys = []
for i in range(0,best-1):
    #print(most_common_keys[0])
    most_common_keys.append(_most_common_keys[i][0])
print(most_common_keys)


In [ ]:
#associamo alle key più frequenti selezionate sopra il rispettivo profitto
most_common_keys_prof = []
for el in most_common_keys:
    most_common_keys_prof.append(key_profits[str(el)])

7. stampiamo ora un istogramma che associa alle keywords più comuni il rispettivo profitto

In [ ]:
fig, ax = plt.subplots()
ax.bar(most_common_keys,most_common_keys_prof)
fig.set_figwidth(27)
fig.set_figheight(13)
plt.xticks(rotation=90,fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("key", fontsize=20)
plt.ylabel("Profits", fontsize=20)
plt.show()

visto che il grafico così è poco leggibile rappresentiamo i risutlati in un altro modo

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
from nltk.tokenize import word_tokenize


plt.subplots(figsize=(12,12))
stop_words=set(stopwords.words('english'))
stop_words.update(',',';','!','?','.','(',')','$','#','+',':','...',' ','')

Movies['keywords'].dropna(inplace=True)
Movies['keywords'] = Movies['keywords'].astype(str)
words=Movies['keywords'].apply(nltk.word_tokenize)
word=[]
for i in words:
    word.extend(i)
word=pd.Series(word)
word=([i for i in word.str.lower() if i not in stop_words])
wc = WordCloud(background_color="black", max_words=4000,stopwords=STOPWORDS, max_font_size= 60,width=1000,height=1000)
wc.generate(" ".join(word))
plt.imshow(wc)
plt.axis('off')
fig=plt.gcf()
fig.set_size_inches(10,10)
plt.show()

si può fare altro, ma ora ci spostiamo su altro...

### qualitiativo 3: paesi di produzione ###

1. Troviamo e salviamo tutti i paesi di produzione del dataframe

In [ ]:
countries=[]
for row in Movies.index:
    _countries = Movies.loc[row,'production_countries']
    for k in _countries:
        if k not in countries:
            countries.append(k)

print(len(countries))
print(countries)

2. definiamo una funzione che dato un pease di produzine calcoli il profitto dei film girati li

In [ ]:
def country_average_profit(country):
    sum = 0
    count = 0
    for row in Movies.index:
        if (country in Movies.loc[row, 'production_countries'] and Movies.loc[row, 'profit_perc'] >= 0):
            sum += Movies.loc[row, 'profit_perc']
            count+=1
    if count != 0:
        return sum/count
    else:
        return -1

3. creiamo una hashmap in cui associamo ad ogni paese di produzione il relativo profitto, mettendo in bad_countries i paesi i cui film non hanno info sufficienti per il calcolo dei profitti

In [ ]:
country_profits={} #declared as hashmap
bad_countries=[] #TODO: si può fare subito set o serve array?
for c in countries:
    q = country_average_profit(c)
    if q != -1:
        country_profits[str(c)]=q
    else:
        bad_countries.append(c)

4. rimuoviamo i paesi di produzione dei cui film non abbiamo info

In [ ]:
set_countries = set(countries)
bad_countries_set = set(bad_countries)
set_countries_cleaned = set_countries - bad_countries_set


print("Numero di paesi di produzione totali:\t\t",len(set_countries))
print("Numero di paesi di produzione da scartare:\t",len(bad_countries_set))
print("Numero di paesi di produzione da analizzare:\t",len(set_countries_cleaned))
print("Numero di profit per paese da analizzare:\t",len(country_profits))


5. mettiamo in un array, nel giusto ordine, i profitti da plottare

In [ ]:
country_profits_plot = []
for el in set_countries_cleaned:
    country_profits_plot.append(country_profits[str(el)])

In [ ]:
fig, ax = plt.subplots()
ax.bar(list(set_countries_cleaned),list(country_profits_plot))
fig.set_figwidth(20)
fig.set_figheight(13)
plt.xticks(rotation=90,fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("key", fontsize=20)
plt.ylabel("Profits", fontsize=20)
plt.show()

6. Notiamo un valore molto alto per la Jamaica, ci stupisce. Controlliamo quanti e quali film ci sono

In [ ]:
for row in Movies.index:
    if 'JM' in Movies.loc[row,'production_countries']:
        print(Movies.loc[row,'profit_perc'])

solo due, dobbiamo fare lo stesso lavoro fatto per le keywords sulla frequenza

In [ ]:
#TODO: fare country con frequenza

******************************************************************************************************************************************
******************************************************************************************************************************************

## Logaritmi, decidere dove va fatto!!##

purtroppo la distrubuzione dei profitti è pessima e potrebbe portare a delle rpoblematiche future, per migliorarla possiamo applicare la funzione logaritmica

In [ ]:
#TODO: separare in due fasi, prima stampiamo la distribuzione, poi diciamo che fa cagare e dopo applichiamo log e diciamo che va meglio
Movies['log_perc'] = np.log(Movies['profit_perc'])
fig, ax = plt.subplots(figsize = (16, 6))
plt.subplot(1,2,1)
plt.hist(Movies['profit_perc'])
plt.title('Distribution of Profits%')
plt.subplot(1,2,2)
plt.hist(Movies['log_perc'])
plt.title('Log Distribution of Profits%')
plt.show()



Il grafico sembra dire che il log ha funzionato, controliamo la skewness

In [ ]:
print('Skewness : %f '% Movies['profit_perc'].skew())
print('Applied Log Skewness : %f '% Movies['log_perc'].skew())


i risutlati confermano quanto suggerito dal grafico, analizziamo ancora come la distribuzione sia cambiata e quanto effettivamente sia migliorata

In [ ]:
sns.distplot(Movies['profit_perc'])
plt.figure(figsize=(16,6))
plt.legend(['Profit'])
plt.title('Profit Perc')
sns.distplot(Movies['log_perc'])
plt.legend(['Profit'])
plt.title('Log Profit Perc')
plt.show()

miglioratissima....ossia studiamo bene cosa vogliono dire sti grafici

******************************************************************************************************************************************
******************************************************************************************************************************************

## Dataset unito a crew ##

**Iniziamo a lavorare con il dataset contenente il cast e il direttore**

In [ ]:
Movies.columns

In [ ]:
cast = pd.read_csv("tmdb_5000_credits.csv")

In [ ]:
feat_to_manage = ['cast','crew']
for f in feat_to_manage:
    cast[f] = cast[f].apply(literal_eval)

#Two functions that convert directors and actors from json to list-str
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

def get_actors(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        
        return names
    return []

In [ ]:
#Create two new column correctly formatted
cast['director'] = cast['crew'].apply(get_director)
cast['actors'] = cast['cast'].apply(get_actors)

In [ ]:
#Drop old columns
cast.drop('cast',inplace=True,axis=1)
cast.drop('crew',inplace=True,axis=1)
cast.drop('title',inplace=True,axis=1)



In [ ]:
Movies.columns

In [ ]:
#rename Movie_id to id, preparing for the merge
cast = cast.rename(columns={'movie_id': 'id'})

#Merge two dataframe Movies,cast
full_df = pd.merge(Movies,cast,on="id",how="inner")


In [ ]:
full_df

Let's see how many times actors appears in different films

In [ ]:

actors=[]


for i in full_df['actors']:
    actors.extend(i)

actors = list(filter(None, actors))


plt.subplots(figsize=(12,10))
ax=pd.Series(actors).value_counts()[:15].sort_values(ascending=True).plot.barh(width=0.9,color=sns.color_palette('inferno_r',40))
for i, v in enumerate(pd.Series(actors).value_counts()[:15].sort_values(ascending=True).values): 
    ax.text(.8, i, v,fontsize=10,color='white',weight='bold')

plt.title('Actors with highest appearance')
ax.patches[14].set_facecolor('r')
plt.show()


### Registi ##

In [ ]:
directors=[]


for i in full_df['director']:
    directors.append(i)

directors = list(filter(None, directors))


plt.subplots(figsize=(12,10))
ax=pd.Series(directors).value_counts()[:34].sort_values(ascending=True).plot.barh(width=0.9,color=sns.color_palette('inferno_r',40))
for i, v in enumerate(pd.Series(directors).value_counts()[:34].sort_values(ascending=True).values): 
    ax.text(.8, i, v,fontsize=10,color='white',weight='bold')

plt.title('Directors with highest appearance')
ax.patches[14].set_facecolor('r')
plt.show()

Calcoliamo Score medio per i registi più presenti(>= 10)

In [ ]:
#Filter the directors with made films >= 10,then calculate the mean scores
#Dovrebbe fare lo stesso lavoro di score_for_director applicato poi alla hashmap, quindi ho rimosso entrambe

director_group = full_df.groupby('director').filter(lambda x : len(x) >= 10)
mean_scores = director_group.groupby('director')['score'].mean().sort_values(ascending=False).reset_index(name="score")
mean_scores

In [ ]:
fig, ax = plt.subplots()
ax.bar(mean_scores['director'],mean_scores['score'])
fig.set_figwidth(20)
fig.set_figheight(13)
plt.xticks(rotation=90,fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel("Directors", fontsize=20)
plt.ylabel("Scores", fontsize=20)
plt.show()

cerchiamo non rating ma score: Fatto ^

******************************************************************************************************************************************
******************************************************************************************************************************************

## Dati quantitativi ##

In [ ]:
#Get only the first genre of each films,
full_df['genres'] = full_df.apply(lambda x: x['genres'][0] if len(x['genres'])>0 else None, axis=1)

In [ ]:
full_df.head(10)

******************************************************************************************************************************************
******************************************************************************************************************************************

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

## random forest ##

droppare i nan
mettere in codifica hot il primo elemento delle colonne colonne:
    genere
    keyword
    director
    attori
    paese produ
    
_____________________________________________________________________
fare random forest che predice score/popul/rev/profit/altro usando le colonne fatte sopra
    

******************************************************************************************************************************************
******************************************************************************************************************************************

#TOTO: 

Toccherà caccaire nel cesso le hashmap

non è che tocca lavorare sui profitti ottenuti col logaritmo?

trovare modo per selezionare solo le country utili (con abbastanza film)

#mettere al posto dei -1 i NaN ma solo nella singola colonna non su tutta la riga
 
Qualitativi --> plot e contare, no correlazioni

sarebbe carino trovare sia le key con più profitto tra le più comuni sia le più comuni tra quelle con più profitto

#Eliminare valori inutile e insensati anche per revenue e "segnarli"
#Ricalcolare profit selezinando solo i film con valori sensati (!=-1)

Plottare profitto in base a:
    #genere
    keyword
    paese prod
    lingua originale (forse)

Plottare count dei generi in per ogni paese di prod (ad esempio giappone potrebbe avere più animazione della GB)

\************************************************

Dati quantitativi --> matematica:

Provare corr tra profitto(non revenue) e budget 
Provare corr tra profitto(non revenue) e score 
Provare corr tra profitto(non revenue) e popularity 

.
.
.
.

usiamo tutte le verità scoperte per fare varie predizioni
